In [1]:
import win32com.client
import pythoncom
import sys
import pandas as pd # pandas 사용을 위해서는 numpy 라이브러리 필요

In [2]:
# XASession : 서버 연결, 로그인을 위한 클래스
# XASessionEvents : XASession과 관련된 이벤트(메시지)를 받기 위한 클래스
class XASessionEvents:
    login_state = False
    
    # 로그인 결과를 사용자에게 통지하기 위한 메서드(이름 변경X)
    def OnLogin(self, code, msg):
        print ("OnLogin : ", code, msg)
        XASessionEvents.login_state = True    

In [3]:
def Login(url='hts.ebestsec.co.kr', port=200001, svrtype=0, id='userid', pwd='password', cert='cert_password'):
    # XASession 클래스(XASession + XASessionEvents)에 대한 인스턴스 생성
    session = win32com.client.DispatchWithEvents("XA_Session.XASession", XASessionEvents)
    session.SetMode("_XINGAPI7_", "TRUE")
    result = session.ConnectServer(url, port)
    
    # 연결이 정상적으로 되면 result 는 True를 반환
    if not result:
        nErrCode = session.GetLastError()
        strErrMsg = session.GetErrorMessage(nErrCode)
        return (False, nErrCode, strErrMsg, None, session)
    
    # 로그인을 위해서는 사용자 아이디, 비밀번호, 공인인증서 비밀번호 필요
    session.Login(id, pwd, cert, svrtype, 0)
    
    # 로그인 요청에 대한 결과를 받을 때까지 대기
    while XASessionEvents.login_state == False:
        pythoncom.PumpWaitingMessages()
    
    # 내 계좌의 개수 확인
    accounts = []
    num_accounts = session.GetAccountListCount()
    
    for i in range(num_accounts):
        accounts.append(session.GetAccountList(i))
        
    return (True, 0, "OK", accounts, session)    

In [4]:
if __name__ == "__main__":
    # passwords.csv 파일 안에 계좌에 대한 정보를 작성
    account_info = pd.read_csv("passwords.csv", converters={'account_num':str}) # DataFrame
    transaction_info = account_info.query("type=='transaction'") # DataFrame (one row)
    if len(transaction_info) > 0:
        account_num = transaction_info['account_num'].values[0].strip() # Value of Series
        id = transaction_info['user_id'].values[0].strip() # Value of Series
        pwd = transaction_info['pwd'].values[0].strip() # Value of Series
        cert = transaction_info['cert_pwd'].values[0].strip() # Value of Series
        url = transaction_info['url'][0].strip() # Value of Series
        
        result, code, msg, accounts, session = Login(url="hts.ebestsec.co.kr", port=200001, svrtype=0, id=id, pwd=pwd, cert=cert)
        
        # 정상적으로 로그인이 안됐으면 프로그램 종료
        if result == False:
            sys.exit(0)
            
        for i in accounts:
            print (i)
            
    else:
        print ("check the file passwords.csv")

OnLogin :  2003 [공인인증] Full 인증과정에서 다음과 같은 오류가 발생하였습니다.
인증서가 없습니다.
